<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/%D0%9D%D0%B5%D0%BE%D0%B4%D0%BD%D0%BE%D0%B7%D0%BD%D0%B0%D1%87%D0%BD%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import adagram
import spacy
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from sklearn.metrics.pairwise import cosine_distances
from nltk.stem import WordNetLemmatizer 

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words


def opt_norm(words):
    uniq = list(set(words))
    norm_uniq = {word:morph.parse(word)[0].normal_form for word in uniq}
    norm_words = [norm_uniq.get(word) for word in words]
    norm_words = [word for word in norm_words if word]
    return norm_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


Обучаем адаграм

In [0]:
corpus = open('corpus_ng.txt', encoding = 'utf-8').read()

In [0]:
norm_corp = opt_norm(tokenize(corpus))

In [0]:
f = open('corpus.txt', 'w', encoding = 'utf-8')
f.write(' '.join(norm_corp))
f.close()

In [0]:
!adagram-train corpus.txt mod1.pkl --dim 300

[INFO] 2020-02-28 11:19:32,850 Building dictionary...
[INFO] 2020-02-28 11:30:00,700 Done! 25010 words.
[INFO] 2020-02-28 11:40:35,728 1.24% -8.3443 0.0247 1.1/2.0 0.10 kwords/sec
[INFO] 2020-02-28 11:40:49,089 2.49% -8.2633 0.0244 1.1/2.0 4.79 kwords/sec
[INFO] 2020-02-28 11:41:01,878 3.73% -8.1649 0.0241 1.1/2.0 5.00 kwords/sec
[INFO] 2020-02-28 11:41:14,301 4.97% -8.0576 0.0238 1.1/2.0 5.15 kwords/sec
[INFO] 2020-02-28 11:41:26,021 6.22% -7.9466 0.0234 1.1/2.0 5.46 kwords/sec
[INFO] 2020-02-28 11:41:37,613 7.46% -7.8346 0.0231 1.1/3.0 5.52 kwords/sec
[INFO] 2020-02-28 11:41:48,974 8.70% -7.7229 0.0228 1.2/3.0 5.63 kwords/sec
[INFO] 2020-02-28 11:42:00,271 9.95% -7.6131 0.0225 1.2/3.0 5.67 kwords/sec
[INFO] 2020-02-28 11:42:11,639 11.19% -7.5062 0.0222 1.2/3.0 5.63 kwords/sec
[INFO] 2020-02-28 11:42:22,803 12.43% -7.4029 0.0219 1.3/4.0 5.73 kwords/sec
[INFO] 2020-02-28 11:42:34,265 13.68% -7.3039 0.0216 1.3/4.0 5.58 kwords/sec
[INFO] 2020-02-28 11:42:45,640 14.92% -7.2098 0.0213 1.3/

In [0]:
vm = adagram.VectorModel.load("mod1.pkl")

## Адаграм в определении перефразирования

In [0]:
data = pd.read_csv('paraphraser_norm.csv')
data.head(2)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешать стрелять на поражение по...,полиция мочь разрешать стрелять по хулиган с т...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский на проникновение в жилище ре...,правило внесудебный проникновение полицейский ...


In [0]:
def get_words_in_context(words, window=3):
    contexts = []
    for i, word in enumerate(words):
        word_and_cont = [word]
        word_cont = []
        for j in range(i-window, i+window+1):
            if j in range(0, len(words)) and j != i:
                word_cont.append(words[j])
        word_and_cont.append(word_cont)
        contexts.append(word_and_cont)
    return contexts   

In [0]:
def dis_get_embedding(text, model, dim):
    words = text.split()
    word2context = get_words_in_context(words, 3)
    vectors = np.zeros((len(word2context), dim))
    for i, (word, context) in enumerate(word2context):
        try:
            num = model.disambiguate(word, context).argmax()
            vec = model.sense_vector(word, num)
            vectors[i] = vec
        except (KeyError, ValueError):
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

In [0]:
dim = 300
X_text_1 = np.zeros((len(data['text_1_norm']), dim))
X_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1[i] = dis_get_embedding(text, vm, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2[i] = dis_get_embedding(text, vm, dim)

In [0]:
X_text = np.concatenate([X_text_1, X_text_2], axis=1)
y = data['label'].values

In [0]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced')

In [0]:
scores = cross_val_score(clf, X_text, y, cv=10, scoring='f1_macro')

In [0]:
print('Macro f1-score')
for score in scores:
    print(score)
print('\nMean score')
print(np.mean(scores))

Macro f1-score
0.44059492105877524
0.42692947690192756
0.4631405353209865
0.47681058107372326
0.4549289538300528
0.43898328018701854
0.37050355401342916
0.3174813419357707
0.31467056043142044
0.39399778358106735

Mean score
0.4098040988334172


## Алгоритм Леска

In [0]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [0]:
def lesk(word, context):
    defs = [set(synset.definition().split()) for synset in wn.synsets(word)]
    inters = [len(context.intersection(one_def)) for one_def in defs]
    number = inters.index(max(inters))
    return number    

In [0]:
umbig_words = 0
right_res = 0
for sent in corpus_wsd:
    clean_sent = [word[1] for word in sent]
    word2context = get_words_in_context(clean_sent, 3)
    for i, word in enumerate(sent):
        if word[0]:
            umbig_words += 1
            context = set(word2context[i][1])
            best = lesk(word[1], context)
            if wn.synsets(word[1])[best] == wn.lemma_from_key(word[0]).synset():
                right_res += 1

In [0]:
right_res/umbig_words

0.3839308415967455

Для того чтобы улучшить алгоритм, попробуем вместо пересечения множеств учитывать косинусное расстояние между векторами контекста и возможных определений слова

In [0]:
model = gensim.models.KeyedVectors.load_word2vec_format('model1.bin', binary=True)

In [0]:
model.most_similar('good')

[('bad', 0.7223736047744751),
 ('excellent', 0.7079846858978271),
 ('decent', 0.6534404754638672),
 ('poor', 0.5662919282913208),
 ('better', 0.56507408618927),
 ('nice', 0.5626944303512573),
 ('luck', 0.550661563873291),
 ('wonderful', 0.5491265058517456),
 ('fortunate', 0.5324512720108032),
 ('pretty', 0.5311379432678223)]

In [0]:
def get_embedding(text, model, dim):
    vectors = np.zeros((len(text), dim))
    for i,word in enumerate(text):
        try:
            v = model[word]
            vectors[i] = v
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [0]:
def en_tokenize(text):
    words = [word.strip(punct).lower() for word in text.split()]
    words = [word for word in words if word]
    return words

In [0]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def spacy_lemm(sent):
  sent = ' '.join(tokenize(sent))
  doc = nlp(sent)
  out_sent = [w.lemma_  for w in doc if w.lemma_ !='-PRON-']
  return(out_sent)

In [0]:
umbig_words = 0
right_res = 0
for sent in corpus_wsd[:100]:
    clean_sent = [word[1] for word in sent]
    word2context = get_words_in_context(clean_sent, 3)
    for i, word in enumerate(sent):
        if word[0]:
            umbig_words += 1
            context = ' '.join(word2context[i][1])
            norm_context = spacy_lemm(context)
            context_vec = get_embedding(norm_context, model, 300)
            defs = [get_embedding(spacy_lemm(synset.definition()), model, 300) for synset in wn.synsets(word[1])]
            dist = [cosine_distances([context_vec], [def_vec]) for def_vec in defs]
            number = dist.index(min(dist))
            if wn.synsets(word[1])[number] == wn.lemma_from_key(word[0]).synset():
                right_res += 1

In [0]:
right_res/umbig_words

0.4252577319587629

На больших объемах очень долго считается, возможно что на всем корпусе результат окажется примерно таким же как в простом алгоритме Леска